### 运行环境

Version：Python 3
Website：www.douban.com

### 通过网站归类标签中“个人管理”类，获取书籍基础信息

In [1]:
# coding=utf-8

import requests
from bs4 import BeautifulSoup
import urllib


url='http://www.douban.com/tag/' + urllib.request.quote('个人管理') + '/book?start=0'
html = requests.get(url)
soup = BeautifulSoup(html.content,'lxml')

class_books = soup.find('div', {'class': 'mod book-list'})
books = class_books.find_all('dd')
for book_info in books:
    title = book_info.find('a', {'class':'title'})
    print(title.get_text())
    desc = book_info.find('div', {'class':'desc'}).get_text()
    print(desc.strip())
    break

少有人走的路
[美] M·斯科特·派克 / 于海生 / 吉林文史出版社 / 2007-1 / 26.00元


### 统计“小说”标签类的书籍总量

In [4]:
# coding=utf-8

import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import urllib


#Some User Agents
hds=[{'User-Agent':'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'},\
{'User-Agent':'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.12 Safari/535.11'},\
{'User-Agent': 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Trident/6.0)'}]


page_number = 0
page_counter = 0
try_times = 0
flag = True
tag = '新书'
url_base = 'https://book.douban.com/tag/'


while(flag): 
    url= url_base + urllib.request.quote(tag) + '?start=' + str(page_number*20) + '&type=T'

    time.sleep(np.random.rand()*5)
    
    html = requests.get(url, headers=hds[page_number%len(hds)])
    soup = BeautifulSoup(html.content,'lxml')

    class_books = soup.find('ul', {'class': 'subject-list'})
    books = class_books.find_all('li')
    
    if books==None and try_times>5:
        try_times+=1;
        continue
    elif len(books)<=1 or books==None:
        break

    page_number += 1
    try_times = 0
    
    
print('书籍数量：', page_number*20)
    

书籍数量： 1000


### 创建douban-books数据库
CREATE DATABASE IF NOT EXISTS douban_books DEFAULT CHARSET = utf8 COLLATE = utf8_general_ci;

### 创建tag_novel表
USE douban_books;

CREATE TABLE IF NOT EXISTS `tag_novel`(
   `novel_title` VARCHAR(100) NOT NULL,
   `novel_author` VARCHAR(20),
   `novel_publishing` VARCHAR(100),
   `novel_press_date` DATE,
   `novel_price` VARCHAR(100),
   `novel_img_website` VARCHAR(500),
   PRIMARY KEY ( `novel_title` )
)ENGINE=InnoDB DEFAULT CHARSET=utf8;

### 数据库连接

In [6]:
import mysql.connector as mariadb

mariadb_connection = mariadb.connect(
    host = 'localhost',
    user = 'root',
    password = '',
    database = 'douban_books'
)

cursor = mariadb_connection.cursor()

print(mariadb_connection)

### 插入标签“小说”类书籍信息到MySQL

In [8]:
# coding=utf-8

import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import urllib
import mysql.connector as mariadb


# Some User Agents
hds=[{'User-Agent':'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'},\
{'User-Agent':'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.12 Safari/535.11'},\
{'User-Agent': 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Trident/6.0)'}]

# open a connection of mariadb
mariadb_connection = mariadb.connect(
    host = 'localhost',
    user = 'root',
    password = '',
    database = 'douban_books'
)

# get mariadb cursor
cursor = mariadb_connection.cursor()

page_number = 0
page_counter = 0
try_times = 0
flag = True
tag = '新书'
url_base = 'https://book.douban.com/tag/'


while(flag): 
    url= url_base + urllib.request.quote(tag) + '?start=' + str(page_number*20) + '&type=T'
    
    time.sleep(np.random.rand()*5)
    
    html = requests.get(url, headers=hds[page_number%len(hds)])
    soup = BeautifulSoup(html.content,'lxml')

    class_books = soup.find('ul', {'class': 'subject-list'})
    books = class_books.find_all('li')
    
    if books==None and try_times>5:
        try_times+=1;
        continue
    elif len(books)<=1 or books==None:
        break
    # end if 
        
    for book_info in books:
        info = book_info.find('div', {'class': 'info'})
        title = info.find('a')['title'].strip()
        pub = info.find('div', {'class': 'pub'}).get_text().strip()
        list_pub = pub.split('/')


        if len(list_pub) == 4:
            author = list_pub[0].strip()
            publishing = list_pub[1].strip()
            press_year = list_pub[2].strip()
            price = list_pub[3].strip()
        # end if

        img = book_info.find('img')['src']
        # run error
        #preface = book_info.find('p').get_text().strip()
        
        # insert information to mariadb
        try:
            cursor.execute(
                "INSERT INTO tag_novel ( \
                    novel_title, \
                    novel_author, \
                    novel_publishing, \
                    novel_press_date, \
                    novel_price, \
                    novel_img_website \
                ) \
                VALUES (%s, %s, %s, %s, %s, %s)", 
                ( 
                    title,
                    author,
                    publishing,
                    press_year,
                    price,
                    img
                )
            )
        except mariadb.Error as error:
            print("Error: {}".format(error))
            continue
        # end try

    # end for

    page_number += 1
    try_times = 0
    
    print(page_number)
    
# end while

# mariadb transaction commit
mariadb_connection.commit()

# close a connection of mariadb
mariadb_connection.close()
    


1
2
3
4
5
Error: 1062 (23000): Duplicate entry '魔姬 第一卷 觉醒' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '遇到你是我的小确幸' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '代数基本概念' for key 'PRIMARY'
6
Error: 1062 (23000): Duplicate entry '文明的历史' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '怎样读经典' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '元素的盛宴' for key 'PRIMARY'
7
Error: 1062 (23000): Duplicate entry '萧红传' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '况且况且况' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '洛克菲勒写给儿子的38封信' for key 'PRIMARY'
8
Error: 1062 (23000): Duplicate entry '学前教育：从蒙台梭利到瑞吉欧' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '作为一种思想操练的五四' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '父酬者' for key 'PRIMARY'
9
Error: 1062 (23000): Duplicate entry '闪击英雄' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '曾经多想变成你' for key 'PRIMARY'
10
Error: 1062 (23000): Duplicate entry '演说心理学' for key 'PRIMARY'
Error

Error: 1062 (23000): Duplicate entry '自由的迷思' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry 'Teen Titans/Outsiders: The Insiders' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '战争史研究' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '一九八四' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '误读' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '宫廷文化' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '棚户区' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '法律的概念' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '马歇尔' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry 'In Search of Prosperity' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '魔山' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '剑桥插图中世纪史' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '东京奇谭集' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry 'Reinventing the Bazaar' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry 'Cry to Heaven' for key '

Error: 1062 (23000): Duplicate entry 'Private Wars (Queen and Country)' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry 'Absolute Batman' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry 'Lex Luthor' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry 'Batman' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry 'Teen Titans Vol.5: Life and Death' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry 'John Constantine, Hellblazer' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry 'War Before Civilization' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry 'American Ways' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry 'Is Voting for Young People? With a Postscript on Citizen Engagem' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry 'Accountability without Democracy' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry 'Conscience of a Conservative' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '当代分析哲学' for key 'PRIMARY'
Error: 1062 (2

Error: 1062 (23000): Duplicate entry 'A Time to Heal' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '传习录' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '美国核战略' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '长银新生' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '脏话文化史' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '青灯' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '故宫建筑内檐装修' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '情感营销' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '民主转型与巩固的问题' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '古希腊星象说' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '马歇尔传：1945-1959' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '苏格拉底问题与现代性' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '艾森豪威尔传' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '国家引导的发展' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '布鲁克林的荒唐事' for key 'PRIMARY'
Error: 1062 (23000): Dup

Error: 1062 (23000): Duplicate entry 'Superman' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry 'As I Saw It' for key 'PRIMARY'
Error: 1062 (23000): Duplicate entry '万物' for key 'PRIMARY'
50


### For Example: 使用XML-RPC with python

In [4]:
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.methods.users import GetUserInfo

wp = Client('http://localhost/wordpress-magazine/xmlrpc.php', 'admin', 'admin')
wp.call(GetPosts())

wp.call(GetUserInfo())

post = WordPressPost()
post.title = 'My new title'
post.content = 'This is the body of my new post.'
post.terms_names = {
    'post_tag': ['test', 'firstpost'],
    'category': ['Introductions', 'Tests']
}
wp.call(NewPost(post))

'5361'

### 带有特色图像缩略图的发布文章

In [10]:
#coding:utf-8


from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods import posts
from wordpress_xmlrpc.methods import taxonomies
from wordpress_xmlrpc import WordPressTerm
from wordpress_xmlrpc.compat import xmlrpc_client
from wordpress_xmlrpc.methods import media, posts


wp = Client('http://localhost/wordpress-magazine/xmlrpc.php', 'admin', 'admin')

filename = './s28173928.jpg' #上传的图片文件路径
# prepare metadata
data = {
        'name': 's28173928.jpg',
        'type': 'image/jpeg',  # mimetype
}
# read the binary file and let the XMLRPC library encode it into base64
with open(filename, 'rb') as img:
        data['bits'] = xmlrpc_client.Binary(img.read())
response = wp.call(media.UploadFile(data))
# response == {
#       'id': 6,
#       'file': 'picture.jpg'
#       'url': 'http://www.example.com/wp-content/uploads/2012/04/16/picture.jpg',
#       'type': 'image/jpeg',
# }

attachment_id = response['id']
post = WordPressPost()
post.title = '1942—1945：我的上海沦陷生活'
post.content = '1942—1945：我的上海沦陷生活，颜滨 著、采金 整理，人民出版社，59.00元'
post.post_status = 'publish'  #文章状态，不写默认是草稿，private表示私密的，draft表示草稿，publish表示发布
post.terms_names = {
    'post_tag': ['Novel', 'firstpost'], #文章所属标签，没有则自动创建
    'category': ['Books', 'Tests'] #文章所属分类，没有则自动创建
 }
post.thumbnail = attachment_id #缩略图的id
post.id = wp.call(posts.NewPost(post))

In [4]:
import sys

sys.path

['C:\\Users\\xuzew\\Documents',
 'C:\\Users\\xuzew\\Anaconda3\\python37.zip',
 'C:\\Users\\xuzew\\Anaconda3\\DLLs',
 'C:\\Users\\xuzew\\Anaconda3\\lib',
 'C:\\Users\\xuzew\\Anaconda3',
 '',
 'C:\\Users\\xuzew\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\xuzew\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\xuzew\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\xuzew\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\xuzew\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\xuzew\\.ipython']

### 下载网上图片

In [9]:
import urllib.request

urllib.request.urlretrieve("https://img1.doubanio.com/view/subject/m/public/s28173928.jpg", "s28173928.jpg")


('s28173928.jpg', <http.client.HTTPMessage at 0x22c95a94f98>)

### 克隆豆瓣书籍英国小说下的书籍到Wordpress

In [ ]:
# coding=utf-8

import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import urllib


# Some User Agents
hds=[{'User-Agent':'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'},\
{'User-Agent':'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.12 Safari/535.11'},\
{'User-Agent': 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Trident/6.0)'}]

page_number = 0
page_counter = 0
try_times = 0
flag = True
tag = '小说'
url_base = 'https://book.douban.com/tag/'


while(flag): 
    url= url_base + urllib.request.quote(tag) + '?start=' + str(page_number*20) + '&type=T'
    
    time.sleep(np.random.rand()*5)
    
    html = requests.get(url, headers=hds[page_number%len(hds)])
    soup = BeautifulSoup(html.content,'lxml')

    class_books = soup.find('ul', {'class': 'subject-list'})
    books = class_books.find_all('li')
    
    if books==None and try_times>5:
        try_times+=1;
        continue
    elif len(books)<=1 or books==None:
        break
    # end if 
        
    for book_info in books:
        info = book_info.find('div', {'class': 'info'})
        title = info.find('a')['title'].strip()
        pub = info.find('div', {'class': 'pub'}).get_text().strip()
        list_pub = pub.split('/')


        if len(list_pub) == 4:
            author = list_pub[0].strip()
            publishing = list_pub[1].strip()
            press_year = list_pub[2].strip()
            price = list_pub[3].strip()
        # end if

        img = book_info.find('img')['src']
        # run error
        #preface = book_info.find('p').get_text().strip()
        
        # insert information to mariadb
        try:
            cursor.execute(
                "INSERT INTO tag_novel ( \
                    novel_title, \
                    novel_author, \
                    novel_publishing, \
                    novel_press_date, \
                    novel_price, \
                    novel_img_website \
                ) \
                VALUES (%s, %s, %s, %s, %s, %s)", 
                ( 
                    title,
                    author,
                    publishing,
                    press_year,
                    price,
                    img
                )
            )
        except mariadb.Error as error:
            print("Error: {}".format(error))
            continue
        # end try

    # end for

    page_number += 1
    try_times = 0
    
    print(page_number)
    
# end while

    